# Entrenamiento y evaluación de modelo experimento 1
* **Autor:** Julian Zuloaga
* **Asignatura:** Memoria de Título
* **Fecha:** 01 de agosto de 2022
---

**Descripción:** Este notebook de Colab se encarga de tomar el fragmento de la base de datos AffectNet procesada para contar con anotaciones en formato YOLOv4 (mediante el programa del notebook "Software para generación de anotaciones en formato YOLOv4 | MDT.ipynb") y luego entrenar un modelo de YOLOv4 con ella (YOLOv4_AffectNet_v2).

* **Nota: Se recomienda hacer una copia de este cuaderno en su unidad de Google Drive personal antes de ejecutar las celdas de código.**
---

## Paso 1: Se importan recursos

In [ ]:
# Se clona repositorio con Darknet y YoloV4
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Se modifica archivo Makefile para habilitar uso de GPU y OPENCV
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# Para pruebas sobre CPU
#%cd darknet
#!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
#!sed -i 's/CPU=0/CPU=1/' Makefile

In [ ]:
# Se ejecuta comando make de darknet para poder usar el archivo ejecutable para entrenamiento o detecciones
!make

In [ ]:
# Se definen funciones auxiliares
# Para mostrar imágenes en Colab
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
# Para descargar archivos desde Colab al PC local
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# Se monta unidad de Google Drive para transferir archivos más rápido
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Se creal symbolic link con dirección de la unidad de Drive para facilitar sintaxis.
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# Se retorna a directorio de darknet
%cd darknet

In [ ]:
# se copian los datasets para entrenamiento y evaluación a la CloudVM de Colab
!cp /mydrive/YOLOv4_AffectNet_v2/obj.zip ../
!cp /mydrive/YOLOv4_AffectNet_v2/test.zip ../

In [ ]:
# se descomprimen los archivos de la base de datos y se guardan en la carpeta data de darknet
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

In [ ]:
# Se sube el archivo de configuración de la red de YOLOv4 a la CloudVM de Colab
!cp /mydrive/YOLOv4_AffectNet_v2/yolov4-obj.cfg ./cfg

In [ ]:
# Se cargan más archivos de configuración necesarios a la CloudVM de Colab
!cp /mydrive/YOLOv4_AffectNet_v2/obj.names ./data
!cp /mydrive/YOLOv4_AffectNet_v2/obj.data  ./data
!cp /mydrive/YOLOv4_AffectNet_v2/generate_train.py ./
!cp /mydrive/YOLOv4_AffectNet_v2/generate_test.py ./

In [ ]:
# Se ejecutan estos scripts de python necesarios para el entrenamiento
!python generate_train.py
!python generate_test.py

In [ ]:
# antes de inicial el entrenamiento, está fuertemente recomendado copiar estos pesos pre entrenados de YOLO para hacer
# transferlearning y así, que el entrenamiento sea más corto.
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

## Paso 2: Se inicia entrenamiento del modelo de YOLOv4

In [ ]:
# Se inicia entrenamiento del modelo
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

## Paso 3: Se reanuda entrenamiento del modelo de YOLOv4
En caso de que el entrenamiento del paso 2 haya sido detenido, es posible retomar el entrenamiento con la siguiente instrucción.

In [ ]:
# retomar entrenamiento desde el último peso guardado
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/YOLOv4_AffectNet_v2/backup/yolov4-obj_last.weights -dont_show

## Paso 4: Se evalúa la precisión del modelo generado

In [ ]:
# Se evalúa la precisión del modelo generado mediante cálculo de mAP
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/YOLOv4_AffectNet_v2/backup/yolov4-obj_last.weights

## Paso 5: Se realizan pruebas de inferencia sobre imágenes

In [ ]:
# Para realizar pruebas de detección, es necesario cambiar estos parámetros antes del archivo de configuración
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

In [ ]:
# Se ejecuta la detección con imagen guardada en unidad de Google Drive
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/YOLOv4_AffectNet_v2/backup/yolov4-obj_last.weights /mydrive/YOLOv4_AffectNet_v1/imagenes/14.jpg -thresh 0.3
imShow('predictions.jpg')

In [ ]:
# Volver a estos parámetros de configuración para retomar entrenamiento
%cd cfg
!sed -i 's/batch=1/batch=64/' yolov4-obj.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' yolov4-obj.cfg
%cd ..

Fin del programa.